In [1]:
import sys
sys.path.append('../../src/')

import pandas as pd
import numpy as np
import networkx as nx
import scipy.sparse as sp
from stellargraph import StellarDiGraph

from extract_network import extract_network

2021-12-15 11:32:19.329569: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-15 11:32:23.596683: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 557 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:18:00.0, compute capability: 7.5
2021-12-15 11:32:23.598545: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13524 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:19:00.0, compute capability: 7.5
2021-12-15 11:32:23.600092: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13524 

In [25]:
data = pd.read_csv('./aggXquarter.txt',delimiter = "\t")
folder_name = './'

In [26]:
data

,group,study.quarter,actor,target,number.wins
0,G1,1,bbb,rgn,1
1,G1,1,bgn,bng,2
2,G1,1,bgn,ggg,4
3,G1,1,bgn,grr,1
4,G1,1,bgn,nrb,3
...,...,...,...,...,...
955,G2,4,rnr,ngg,1
956,G2,4,rnr,ngn,1
957,G2,4,rnr,nnb,2
958,G2,4,rnr,nnr,2


In [27]:
data = data.dropna(subset=['number.wins'])

In [28]:
data

,group,study.quarter,actor,target,number.wins
0,G1,1,bbb,rgn,1
1,G1,1,bgn,bng,2
2,G1,1,bgn,ggg,4
3,G1,1,bgn,grr,1
4,G1,1,bgn,nrb,3
...,...,...,...,...,...
955,G2,4,rnr,ngg,1
956,G2,4,rnr,ngn,1
957,G2,4,rnr,nnb,2
958,G2,4,rnr,nnr,2


In [29]:
df = data[['actor', 'target', 'number.wins']]

In [30]:
df.columns = ['source', 'target', 'weight']

In [31]:
graph = StellarDiGraph(edges=df)

In [32]:
print(graph.info())

StellarDiGraph: Directed multigraph
 Nodes: 40, Edges: 960

 Node types:
  default: [40]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [960]
        Weights: range=[1, 39], mean=2.47187, std=2.93219
        Features: none


In [33]:
adj = graph.to_adjacency_matrix(weighted=True)

In [11]:
adj

<40x40 sparse matrix of type '<class 'numpy.int64'>'
	with 421 stored elements in Compressed Sparse Row format>

In [13]:
adj-adj.transpose()

<40x40 sparse matrix of type '<class 'numpy.int64'>'
	with 674 stored elements in Compressed Sparse Row format>

In [14]:
adj.max()

62

In [15]:
adj.min()

0

In [16]:
df

,source,target,weight
0,bbb,rgn,1
1,bgn,bng,2
2,bgn,ggg,4
3,bgn,grr,1
4,bgn,nrb,3
...,...,...,...
955,rnr,ngg,1
956,rnr,ngn,1
957,rnr,nnb,2
958,rnr,nnr,2


In [37]:
node_list = list(graph.nodes())

In [38]:
print(len(node_list))
adj, node_list = extract_network(adj, np.array(node_list), lowest_degree=1)
print(len(node_list))

40
21


In [39]:
np.save(folder_name+'node_list', node_list)
sp.save_npz(folder_name+'adj', adj)